In [ ]:
import numpy as np
from numpy.testing import assert_allclose

from qiskit import QiskitError
from qiskit import QuantumRegister, QuantumCircuit
from qiskit.circuit.library import HGate, QFT

from qiskit.quantum_info.random import random_unitary, random_density_matrix, random_pauli
from qiskit.quantum_info.states import DensityMatrix, Statevector
from qiskit.quantum_info.operators.operator import Operator
from qiskit.quantum_info.operators.symplectic import Pauli, SparsePauliOp

In [ ]:
def rand_vec(n, normalize=False):
    """Return complex vector or statevector"""
    seed = np.random.randint(0, np.iinfo(np.int32).max)
    rng = np.random.default_rng(seed)
    vec = rng.random(n) + 1j * rng.random(n)
    if normalize:
        vec /= np.sqrt(np.dot(vec, np.conj(vec)))
    return vec

def rand_rho(n):
        """Return random pure state density matrix"""
        rho = rand_vec(n, normalize=True)
        return np.outer(rho, np.conjugate(rho))

In [ ]:
print(np.sum([np.abs(a)**2 for a in rand_vec(3, True)]))
print(np.trace(rand_rho(3)))

In [ ]:
"""Test subsystem initialization from N-qubit array."""
# Test automatic inference of qubit subsystems
rho = rand_rho(8)
for dims in [None, 8]:
    state = DensityMatrix(rho, dims=dims)
    assert_allclose(state.data, rho)
    assert state.dim == 8
    assert state.dims() == (2, 2, 2)
    assert state.num_qubits == 3

In [ ]:
"""Test initialization from array."""
rho = rand_rho(3)
state = DensityMatrix(rho)
assert_allclose(state.data, rho)
assert state.dim == 3
assert state.dims() == (3,)
assert state.num_qubits == None

rho = rand_rho(2 * 3 * 4)
state = DensityMatrix(rho, dims=[2, 3, 4])
assert_allclose(state.data, rho)
assert state.dim == 2 * 3 * 4
assert state.dims() == (2, 3, 4)
assert state.num_qubits == None

In [ ]:
"""Test initialization exception from array."""
rho = rand_rho(4)
try:
    DensityMatrix(rho, dims=[4, 2])
except QiskitError:
    pass
try:
    DensityMatrix(rho, dims=[2, 4])
except QiskitError:
    pass
try:
    DensityMatrix(rho, dims=5)
except QiskitError:
    pass